In [3]:
import numpy as np
import collections 
import random as rd


In [64]:
class puzzle:
    NORTH ,SOUTH,EAST ,WEST = 1,2,3,4
    board = None
    parent = None
    depth = None
    neighbours = None
    isMax = False
    
    def __init__(self,board,parent=None,isMax=True):
        self.board = board
        self.parent = parent
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0
        self.isMax = isMax
        self.getNeighbours()
    
    def getMoves(self,move):

        moves = []
        
        
        if move == self.NORTH:
            direction = (-1,0)
            #Recorremos las fichas de arriba hacia abajo
            for i in range(1,4):
                for j in range(4):
                    moves.append([(i,j)])
        
        if move == self.SOUTH:
            direction = (1,0)
            #Recorremos las fichas de abajo hacia arriba
            for i in reversed(range(0,3)):
                for j in range(4):
                    moves.append([(i,j)])
            
        if move == self.EAST:
            direction = (0,1)
            #Recorremos las fichas de izquierda a derecha
            for j in reversed(range(0,4)):
                for i in range(4):
                    moves.append([(i,j)])
                    
        if move == self.WEST:
            direction = (0,-1)
             #Recorremos las fichas de derecha a izquierda
            for j in range(0,4):
                for i in range(4):
                    moves.append([(i,j)])
        
        
        return moves,direction
    
    #Funcion que realiza los merges de los numeros iguales
    def mergeNumbers(self,obj,dest,board):
        
        if board[obj] == board[dest]:
            board[dest] =board[obj] + board[dest]
            board[obj] = 0
            return True
        return False
            
    def movePiece(self,src,dest,board):
        if dest[0] < 0 or dest[1] < 0  or dest[1] > 3 or dest[0] > 3:
            return False
        elif board[dest] != 0:
            return self.mergeNumbers(src,dest,board)
        
        else:
            board[dest] = board[src]
            board[src] = 0
            return True
            
    
    #Funcion que realiza los movimientos de las fichas
    def moveBoard(self,move):
        board = self.board.copy()
        moves,direction = self.getMoves(move)
        
        for mv in moves:
            src = mv[0]

            dest = src
            band = True
            while band:
                dest = (dest[0] + direction[0],dest[1] + direction[1])
                band = self.movePiece(src,dest,board)
                src = dest

        return board
        
    def getNeighbours(self):
        self.neighbours = []
        if self.isMax:
            for i in range(1,5):
                board = self.moveBoard(i)
                #El siguiente if nos permite podar un poco mas los vecinos.
                if not np.array_equal(self.board,board):
                    self.neighbours.append(board)
        else:
            self.getMinNeighbours()
    
    def getMinNeighbours(self):
        for i in range(4):
            for j in range(4):
                if self.board[i,j] == 0:
                    board1 =  self.board.copy()
                    board2 = self.board.copy()
                    board1[i,j] = 2
                    board2[i,j] = 4
                    self.neighbours.append(board1)
                    self.neighbours.append(board2)
                    
    def getPenalty(self):
        penalty = 0
        for i in range(4):
            for j in range(4):
                vecs = [[-1,0],[1,0],[0,-1],[0,1]]
                if self.board[i,j] !=0:
                    for vec in vecs:
                        desti = i+ vec[0]
                        destj = j+ vec[1]
                        if desti >= 0 and desti <= 3 and destj >= 0 and destj <= 3:
                            penalty += abs(self.board[i,j] - self.board[desti,destj] )
        return penalty
    
    
    def getScore(self):
        w = [[0,0,1,3],
             [0,1,3,5],
             [1,3,5,15],
             [3,5,15,30]]
        values = np.sum(self.board * w)

        return np.sum(self.board) + np.count_nonzero(self.board == 0) + values - self.getPenalty()

    def isGoal(self):
        return np.any(self.board == 2048)
    
    

In [65]:
def CreaEstado():
    board = np.zeros((4,4))
    possvalues = [2,4]
    for i in range(2):
        randpos = (rd.randint(0,3),rd.randint(0,3))
        randvalue = rd.randint(0,1)
        if board[randpos] == 0:
            board[randpos] = possvalues[randvalue]
        else:
            i-=1
    
    return puzzle(board)

startObj = CreaEstado()
print startObj.board

[[0. 0. 0. 0.]
 [0. 2. 4. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
def alpha_beta(node,maxdepth,isMax,alpha,beta):

    finalnode = None
    if node.depth == maxdepth or node.isGoal() or len(node.neighbours) == 0:
        return node,node.getScore()
    
    if isMax:
        bestVal = -10000
        for neighbour in node.neighbours:
            
            newNode = puzzle(neighbour,node,False)
            ansnode,value = alpha_beta(newNode,maxdepth,False,alpha,beta)
            bestVal = max(value,bestVal)
            alpha = max(alpha,bestVal)
            
            if bestVal == value:
                finalnode = ansnode
            
            if beta <= alpha:
                break
        return finalnode,bestVal
    else:
        bestVal = 10000
        for neighbour in node.neighbours:
            newNode = puzzle(neighbour,node,True)
            ansnode,value = alpha_beta(newNode,maxdepth,True,alpha,beta)
            
            bestVal = min(value,bestVal)                
            beta = min(beta,bestVal)
            
            if bestVal == value:
                finalnode = ansnode
                
            if beta <= alpha:
                break
        
        return finalnode,bestVal
        
        
        
node,val = alpha_beta(startObj,6,True,-10000,10000)

print node.board,val